## "scientific turk" web bot
parses sources for new articles, translates them into target languages ["es","pt","pl","zh-CN"]. Wraps it up in a simple html document and writes it out to the appropriate directory.

#### TO DO TECHNICAL:

*   save the articles dictionary as a json or something (__done__)
*   *   (dic backup for later design changes) important
*   implement an automatic parser (__done(sd)__)
*   detect and highlight keywords
*   automatic git push (__done__ but pass cache time should be increased)
*   run the script on an aws server
*   maybe an open source grammar rating mechanism?
*   don't send a separate request to the website for every translation (__done__)
*   same pages shouldn't be crawled twice. Even if they are crawled they shouldn't be overwritten. (__done__)
*   implement waiting if requests exhaust limit
*   also implement a timeout. maybe take articles into a queue.
*   re-read SD's [terms](https://www.sciencedaily.com/terms.htm) on reproduction. (especially on images)
*   take the bot 1 directory up so it won't be pushed into the public repo. (__done__)

#### TO DO GRAPHICAL
*   responsive text, currently changes size by page??
*   resizable design (__done__)
*   mobile-friendly design
*   *   [media queries for different device widths](https://stackoverflow.com/questions/16387400/getting-the-right-font-size-on-every-mobile-device)
*   subscribe and social media block


#### POSSIBLE SOURCES
*   [the conversation](https://theconversation.com/uk/republishing-guidelines)
*   *   approval needed for translation
*   

NOTE: google transalte tokens refresh every hour (?)






---



In [23]:
#for colab
!pip install googletrans
!pip install mechanize

## Character limit: 15K

In [24]:
#import os
#cmd = "cd cemreefe.github.io; git pull; cd .."
#os.system(cmd)

In [25]:
!pip3 install -U deep-translator

from deep_translator import GoogleTranslator

def translate_adaptor(text, source, target):
    translator = GoogleTranslator(source=source, target=target)
    return translator.translate(text, src=source, dest=target)

In [26]:
from datetime import date
hoy  = date.today()
d1   = hoy.strftime("%d/%m/%Y")

In [27]:
from googletrans import Translator
import re

translator = Translator()

In [28]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [29]:
def string_strip(s):
    return re.sub(r"[^A-Za-z0-9]", "-", s)

In [30]:
import html
import random
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

def sciencedaily_parse_article(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')

    # Get the headline
    headline = soup.find('h1', class_='headline').get_text()

    # Get the meta
    meta = soup.find('dd', id='abstract').get_text()

    # Get the img address
    img_tag = soup.find('img', class_='center-block')
    imgaddr = 'https://www.sciencedaily.com' + img_tag['src'] if img_tag else f'/img/dummy{random.randrange(20)}.jpeg'

    # Get the img alt
    imgalt = soup.find('div', class_='photo-caption').get_text()
    cred = '\t' + soup.find('div', class_='photo-credit').find('em').get_text()

    # Get the citation
    citation = soup.find('div', id='citation_mla').get_text()

    # Get the article itself and clean it
    article = soup.find('div', id='text').get_text()
    article = article.replace("<p>", "").replace("</p>", "\n")

    # Get the href at the end if it exists
    href_tab = str(soup.find('a', href=True))

    # Decode HTML entities in the article
    article = html.unescape(article)

    return {
        "headline": headline,
        "meta": meta,
        "imgaddr": imgaddr,
        "imgalt": imgalt,
        "imgcredit": cred,
        "citation": citation,
        "article": article,
        "href": href_tab,
    }


In [31]:
def get_translation(article_dictionary, target_language):
    token_in = "(#@)"
    token_out = "（＃@）" if target_language == "zh-CN" else "(# @)"

    # Define the order of keys explicitly for clarity
    keys = ["headline", "meta", "article", "imgalt"]

    # Join the values of the specified keys using the token_in
    big_string = token_in.join(article_dictionary[key] for key in keys)

    # Use a function to handle translation, and handle the manual fix within the translation function
    big_translated_string = translate_and_fix(big_string, target_language, token_out)

    # Split the translated string using the token_out
    translations_array = big_translated_string.split(token_out)

    # Create a dictionary of translations
    translations = {keys[i]: translations_array[i] for i in range(4)}
    translations.update({
        "imgaddr": article_dictionary.get("imgaddr", ""),
        "citation": article_dictionary.get("citation", ""),
        "imgcredit": article_dictionary.get("imgcredit", ""),
        "href": article_dictionary.get("href", ""),
        "org-headline": article_dictionary.get("headline", ""),
        "lang": target_language
    })

    return translations

def translate_and_fix(text, target_language, token_out):
    # Use your translate_adaptor function here to handle translation
    big_translated_string = translate_adaptor(text, source='en', target=target_language)
    
    # Apply the manual fix
    big_translated_string = big_translated_string.replace('(#@)', token_out)
    
    return big_translated_string


In [32]:
# deprecated
def get_language_dictionary(target_language):
    ld = {}
    words = ["scientific turk", "homepage", "archive", "about us", "source:"]
    for word in words:
        print(word)
        ld[word] = translate_adaptor(word, source='en', target=target_language)
    return ld

In [33]:
slogans = [
    "Wissenschaft für alle, überall.",
    "科学无处不在，无处不在。",
    "Ciencia para todos, en todas partes.",
]

In [34]:
from datetime import date
import re

def html_from_dictionary(translated_dictionary, target_language, language_dictionary, elements_dictionary, slogans):
    hoy = date.today()
    d1 = hoy.strftime("%d/%m/%Y")

    # Read the HTML template
    with open(SUBFOLDER + "templates/details.html", 'r') as template_file:
        html = template_file.read()

    # Prepare the article text
    article_text = "<p>" + translated_dictionary["article"].replace("\n\n", "</p>\n\n<p") + "</p>"

    # Define a dictionary of placeholders and their corresponding values
    placeholders = {
        "$$article-title%%": translated_dictionary.get("headline", ""),
        "$$img-alt%%": translated_dictionary.get("imgalt", ""),
        "$$article-meta%%": translated_dictionary.get("meta", ""),
        "$$meta-tag-text%%": article_text[:150],
        "$$source%%": translated_dictionary.get("citation", ""),
        "$$img.jpg%%": translated_dictionary.get("imgaddr", ""),
        "$$article-text%%": article_text,
        "$$home%%": language_dictionary.get("homepage-" + target_language, ""),
        "$$archive%%": language_dictionary.get("archive-" + target_language, ""),
        "$$about%%": language_dictionary.get("about us-" + target_language, ""),
        "$$jp-translation%%": language_dictionary.get("scientific turk-" + target_language, ""),
        "$$taken-from%%": language_dictionary.get("source:-" + target_language, ""),
        "$$author-name%%": language_dictionary.get("author-name-" + target_language, ""),
        "$$author-img-url%%": language_dictionary.get("author-img-" + target_language, ""),
        "$$author-description%%": language_dictionary.get("author-desc-" + target_language, ""),
        "$$slogan1%%": slogans[0] if len(slogans) > 0 else "",
        "$$slogan2%%": slogans[1] if len(slogans) > 1 else "",
        "$$slogan3%%": slogans[2] if len(slogans) > 2 else "",
        "$$homepage-text%%": elements_dictionary.get("hometext-" + target_language, ""),
        "$$target-language%%": target_language,
        "$$imgcredit%%": translated_dictionary.get("imgcredit", ""),
        "$$href%%": translated_dictionary.get("href", ""),
        "-- ": ",",
        "$$date%%": d1,
    }

    # Use a regular expression to find all occurrences of $$<tag>%%
    pattern = r'\$\$([^%]+)%%'
    matches = re.finditer(pattern, html)

    # Replace the matches with their corresponding values or leave them unchanged if no value is found
    for match in matches:
        placeholder = match.group(0)
        value = placeholders.get(placeholder, placeholder)
        html = html.replace(placeholder, value)

    return html


In [35]:
def getnewpath(translated_dict):
  
    from datetime import date
    hoy  = date.today()
    date = hoy.strftime("%Y-%m-%d")
    
    translated_dict["date"]=date
  
    urlheadline = string_strip(translated_dict["org-headline"])+"-"+translated_dict["lang"]
    postsfolder = "/posts/"+date+"/"
    newpathaddr = translated_dict["lang"]+postsfolder
    newheadline = newpathaddr+urlheadline+".html"
    newheadlang = postsfolder+urlheadline+".html"
    
    translated_dict["pathfromhome"]=newheadline
    translated_dict["pathfromlang"]=newheadlang
    
    headlinessofar.append(newheadline)
    return [newheadline,newpathaddr,urlheadline]

In [36]:
SUBFOLDER = ""

In [37]:
import pickle

def save_obj(obj, name ):
    with open(SUBFOLDER+'obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(SUBFOLDER+'obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [38]:
import os

def dic_to_dirfile(dic, target_language, elements_dictionary):
    if target_language == 'en':
        return

    # Translate the dictionary to the target language
    translated_dict = get_translation(dic, target_language)

    # Generate HTML content from the translated dictionary
    html_content = html_from_dictionary(translated_dict, target_language, elements_dictionary)

    # Create a new path for the HTML file
    new_paths = getnewpath(translated_dict)

    # Create the directories recursively
    os.makedirs(os.path.join(SUBFOLDER, new_paths[1]), exist_ok=True)

    # Write the HTML content to the new file
    with open(os.path.join(SUBFOLDER, new_paths[0]), "w+") as file:
        file.write(html_content)

    return [html_content, translated_dict, new_paths[2], new_paths[0]]


In [39]:
import os

def targdic_to_dirfile(tr_di, target_language, elements_dictionary):
    html_content = html_from_dictionary(tr_di, target_language, elements_dictionary)
    new_paths = getnewpath(tr_di)
    
    # Create the directories recursively
    os.makedirs(os.path.join(SUBFOLDER, new_paths[1]), exist_ok=True)

    with open(os.path.join(SUBFOLDER, new_paths[0]), "w+") as file:
        file.write(html_content)

    return [html_content, tr_di, new_paths[2], new_paths[0]]


In [40]:
elements_dictionary = {
    "scientific turk-es":"ciencia para todxs", 
    "scientific turk-pt":"ciência para todos", 
    "scientific turk-tr":"herkes için bilim", 
    "scientific turk-zh-CN":"科学为每个人", 
    "scientific turk-pl":"spokojna gazeta", 
    "scientific turk-de":"friedliche Zeitung", 
    "scientific turk-af":"vreedsame koerant",
    
    "homepage-es":"página principal", 
    "homepage-pt":"pagina inicial", 
    "homepage-tr":"anasayfa",
    "homepage-pl":"strona główna", 
    "homepage-de":"Startseite", 
    "homepage-zh-CN":"主页", 
    "homepage-af":"tuisblad", 
    
    "archive-es":"archivo",
    "archive-pt":"arquivo",
    "archive-tr":"arşiv",
    "archive-zh-CN":"档案",
    "archive-pl":"archiwum",
    "archive-de":"Archiv",
    "archive-af":"argief",
    
    
    "about us-es":"sobre nosotros",
    "about us-pt":"sobre nós",
    "about us-tr":"hakkında",
    "about us-zh-CN":"关于我们",
    "about us-pl":"o nas",
    "about us-de":"Über uns",
    "about us-af":"oor ons",
    
    "source:-es":"fuente:",
    "source:-pt":"fonte:",
    "source:-tr":"kaynakça:",
    "source:-zh-CN":"资源",
    "source:-pl":"źródło",
    "source:-de":"Quelle:",
    "source:-af":"bron:",
    
    "latest-articles-es":"Últimos artículos",
    "latest-articles-pt":"Artigos Mais Recentes",
    "latest-articles-pl":"Ostatnie artykuły",
    "latest-articles-zh-CN":"最新的文章",
    "latest-articles-tr":"En yeni makaleler",
    "latest-articles-de":"Neueste Artikel",
    "latest-articles-af":"Mees onlangse artikels",
    
    "hometext-es":"Bienvenido a Scientific Turk. Le proporcionamos los últimos artículos sobre ciencia y tecnología. Scientific Turk se dedica a la distribución de investigaciones científicas populares en otros idiomas además del inglés.",
    "hometext-pt":"Bem-vindo ao Scientific Turk. Fornecemos os artigos mais recentes sobre ciência e tecnologia. O Scientific Turk é dedicado à distribuição de pesquisas científicas populares em outros idiomas que não o inglês.",
    "hometext-pl":"Witamy w Scientific Turk. Zapewniamy najnowsze artykuły na temat nauki i technologii. Czasopismo Pacifique poświęcone jest rozpowszechnianiu popularnych badań naukowych w językach innych niż angielski.",
    "hometext-zh-CN":"欢迎来到Scientific Turk。 我们为您提供有关科学和技术的最新文章。 Scientific Turk致力于以英语以外的语言分发流行的科学研究。",
    "hometext-tr":"Scientific Turk'e hoş geldiniz. Bilim ve teknoloji ile ilgili en son makaleleri size sunuyoruz. Scientific Turk, popüler bilimsel araştırmaların İngilizce dışındaki dillerde dağıtımına adanmıştır.",
    "hometext-de":"Willkommen im Scientific Turk. Wir präsentieren Ihnen die neuesten Artikel über Wissenschaft und Technologie. Das Scientific Turk widmet sich der Verbreitung populärwissenschaftlicher Forschung in anderen Sprachen als Englisch.",
    "hometext-af":"Welkom by die Scientific Turk. Ons bied die nuutste artikels oor wetenskap en tegnologie aan. Die Scientific Turk is toegewy aan die verspreiding van populêre wetenskaplike navorsing in ander tale as Engels.",
    
    "abouttext-es":"Scientific Turk se estableció para evitar que la barrera del idioma obstruya la accesibilidad de los artículos científicos. En Scientific Turk pensamos que todos los artículos científicos deberían ser fácilmente accesibles para las personas en su propio idioma. Por lo tanto, hemos asumido la responsabilidad de encontrar artículos en Internet de valor decente y traducirlos a tantos idiomas como sea posible, haciéndolos accesibles a muchas más personas de lo que era posible en su idioma original.",
    "abouttext-pt":"O Scientific Turk foi criado para impedir que a barreira do idioma obstrua a acessibilidade de artigos científicos. No Scientific Turk, pensamos que todos os artigos científicos devem ser facilmente acessíveis às pessoas em seu próprio idioma. Portanto, assumimos a responsabilidade de encontrar artigos na Internet de valor decente e traduzi-los para o maior número possível de idiomas, tornando-os acessíveis a muito mais pessoas do que era possível em seu idioma original.",
    "abouttext-pl":"Scientific Turk został utworzony, aby bariera językowa nie utrudniała dostępu do artykułów naukowych. W Scientific Turk uważamy, że wszystkie artykuły naukowe powinny być łatwo dostępne dla ludzi w ich własnym języku. Dlatego podjęliśmy się odpowiedzialności za znalezienie artykułów w Internecie o przyzwoitej wartości i przetłumaczenie ich na jak najwięcej języków, dzięki czemu będą dostępne dla większej liczby osób niż było to możliwe w ich oryginalnym języku.",
    "abouttext-zh-CN":"Scientific Turk的建立是为了防止语言障碍阻碍科学文章的可访问性。 Scientific Turk认为我们所有的科学文章都应该易于人们以自己的语言阅读。 因此，我们承担了在互联网上查找具有体面价值的文章并将其翻译成尽可能多的语言的责任，从而使更多的人可以使用原始语言。",
    "abouttext-tr":"",
    "abouttext-de":"Das Scientific Turk wurde gegründet, um zu verhindern, dass die Sprachbarriere die Zugänglichkeit wissenschaftlicher Artikel behindert. Wir bei Scientific Turk glauben, dass alle wissenschaftlichen Artikel für Menschen in ihrer eigenen Sprache leicht zugänglich sein sollten. Aus diesem Grund haben wir die Verantwortung dafür übernommen, Artikel mit angemessenem Wert im Internet zu finden und in so viele Sprachen wie möglich zu übersetzen, um sie für viel mehr Menschen zugänglich zu machen, als dies in ihrer Originalsprache möglich war.",
    "abouttext-af":"Tydskrif Pacifique is gestig om te voorkom dat die taalversperring die toeganklikheid van wetenskaplike artikels belemmer. In Scientific Turk glo ons dat alle wetenskaplike artikels maklik toeganklik moet wees vir mense in hul eie taal. Daarom het ons die verantwoordelikheid aanvaar om artikels met 'n ordentlike waarde op die internet te vind en dit in soveel tale as moontlik te vertaal, sodat dit vir baie meer mense toeganklik is as wat in hul oorspronklike taal moontlik was.",
    
    "see-more-es":"Ver más",
    "see-more-pt":"Ver mais",
    "see-more-pl":"Zobacz więcej",
    "see-more-zh-CN":"看更多",
    "see-more-tr":"Daha fazla",
    "see-more-de":"Mehr sehen",
    "see-more-af":"Sien meer",
    
    "author-name-es":"Edgardo Terrazas",
    "author-name-pt":"Dário Aguiar",
    "author-name-pl":"Julia Wojnicz",
    "author-name-zh-CN":"Adrian Ng",
    "author-name-tr":"Aygün İncesu",
    "author-name-de":"Kai Schulte",
    "author-name-af":"Imka Friedrich",
    
    "author-desc-es":"Traductor español - inglés en Scientific Turk",
    "author-desc-pt":"Tradutora de português - inglês no Scientific Turk",
    "author-desc-pl":"Tłumacz polsko - angielski w Scientific Turk",
    "author-desc-zh-CN":"Scientific Turk的中英文翻译",
    "author-desc-tr":"Scientific Turk'te Türkçe - İngilizce çevirmen",
    "author-desc-de":"Deutsch - Englisch Übersetzer bei Scientific Turk",
    "author-desc-af":"Afrikaans - Engelse vertaler by Scientific Turk",
    
    "author-img-es":"https://avataaars.io/?avatarStyle=Circle&topType=LongHairCurly&accessoriesType=Blank&hairColor=SilverGray&facialHairType=BeardLight&facialHairColor=Brown&clotheType=GraphicShirt&clotheColor=PastelGreen&graphicType=Diamond&eyeType=WinkWacky&eyebrowType=FlatNatural&mouthType=Disbelief&skinColor=Light",
    "author-img-pt":"https://avataaars.io/?avatarStyle=Circle&topType=WinterHat4&accessoriesType=Blank&hatColor=Red&facialHairType=BeardMedium&facialHairColor=Brown&clotheType=ShirtCrewNeck&clotheColor=Heather&eyeType=Surprised&eyebrowType=DefaultNatural&mouthType=Disbelief&skinColor=Light",
    "author-img-pl":"https://avataaars.io/?avatarStyle=Circle&topType=LongHairBun&accessoriesType=Blank&hairColor=BlondeGolden&facialHairType=Blank&clotheType=ShirtScoopNeck&clotheColor=Pink&eyeType=Cry&eyebrowType=UpDownNatural&mouthType=Serious&skinColor=Pale",
    "author-img-zh-CN":"https://avataaars.io/?avatarStyle=Circle&topType=LongHairNotTooLong&accessoriesType=Kurt&hairColor=Black&facialHairType=Blank&clotheType=BlazerSweater&eyeType=Cry&eyebrowType=Default&mouthType=Smile&skinColor=Light",
    "author-img-tr":"https://avataaars.io/?avatarStyle=Circle&topType=ShortHairShaggyMullet&accessoriesType=Prescription02&hairColor=BrownDark&facialHairType=Blank&clotheType=ShirtScoopNeck&clotheColor=White&eyeType=Close&eyebrowType=FlatNatural&mouthType=Default&skinColor=Light",
    "author-img-de":"https://avataaars.io/?avatarStyle=Circle&topType=LongHairStraightStrand&accessoriesType=Sunglasses&hairColor=BlondeGolden&facialHairType=MoustacheFancy&facialHairColor=Brown&clotheType=BlazerSweater&eyeType=Happy&eyebrowType=AngryNatural&mouthType=Twinkle&skinColor=Light",
    "author-img-af":"https://avataaars.io/?avatarStyle=Circle&topType=LongHairStraight2&accessoriesType=Round&hairColor=PastelPink&facialHairType=Blank&clotheType=ShirtVNeck&clotheColor=Gray01&eyeType=Default&eyebrowType=UpDownNatural&mouthType=Disbelief&skinColor=Black",
}

In [41]:
def url_to_dirfile(url,target_language):

    ar_di = sciencedaily_parse_article(url)

    print(url, ": Article Parsed")

    htmlx = dic_to_dirfile(ar_di,target_language, elements_dictionary)
  
    return [htmlx,ar_di]

In [42]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

def get_article_urls_sd():
    base_url = "https://www.sciencedaily.com"
    user_agent = 'Mozilla/5.0'

    # Send a request to the main page
    req = Request(base_url, headers={'User-Agent': user_agent})
    webpage = urlopen(req).read()
    main_html = webpage.decode("utf-8")

    links = []

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(main_html, 'html.parser')

    # Find all links with the specified format
    for link in soup.find_all('a', href=re.compile(r'/releases/\d+/\d+/\d+\.htm')):
        release_url = link['href']
        headline = link.get_text()
        full_url = base_url + release_url

        links.append((full_url, headline))

    return links


In [43]:
articles = get_article_urls_sd()
            
languages = ["es", "pt", "pl", "zh-CN", "de", "af", 'tr']

In [44]:
articles

[('https://www.sciencedaily.com/releases/2023/10/231005161809.htm',
  'Oldest Fossil Human Footprints in North America Confirmed'),
 ('https://www.sciencedaily.com/releases/2023/10/231005135613.htm',
  'Scientists Discover the Highest Energy Gamma-Rays Ever from a Pulsar'),
 ('https://www.sciencedaily.com/releases/2023/10/231005110728.htm',
  'And Then There Were 6 -- Kinds of Taste, That Is'),
 ('https://www.sciencedaily.com/releases/2023/10/231004105151.htm',
  'Scientists Develop 3D Printing Method That Shows Promise for Repairing Brain Injuries'),
 ('https://www.sciencedaily.com/releases/2023/10/231005161759.htm',
  "New Discovery May 'Unlock' the Future of Infectious Disease and Cancer Treatment"),
 ('https://www.sciencedaily.com/releases/2023/10/231005161752.htm',
  "Brain Is 'Rewired' During Pregnancy to Prepare for Motherhood"),
 ('https://www.sciencedaily.com/releases/2023/10/231005135616.htm',
  'The Medicine of the Future Could Be Artificial Life Forms'),
 ('https://www.scie

In [45]:
#headlinessofar = []
#articlessofar  = {}
#articleurlssofar = []

In [46]:
headlinessofar = load_obj("headlinessofar")
articlessofar = load_obj("articlessofar")
articleurlssofar =load_obj("articleurlssofar")

In [47]:
list(articlessofar.keys())

['Ancient-shell-shows-days-were-half-hour-shorter-70-million-years-ago-es',
 'Ancient-shell-shows-days-were-half-hour-shorter-70-million-years-ago-pt',
 'Ancient-shell-shows-days-were-half-hour-shorter-70-million-years-ago-pl',
 'Ancient-shell-shows-days-were-half-hour-shorter-70-million-years-ago-zh-CN',
 'Why-is-there-any-matter-in-the-universe-at-all--New-study-sheds-light-es',
 'Why-is-there-any-matter-in-the-universe-at-all--New-study-sheds-light-pt',
 'Why-is-there-any-matter-in-the-universe-at-all--New-study-sheds-light-pl',
 'Why-is-there-any-matter-in-the-universe-at-all--New-study-sheds-light-zh-CN',
 'Exoplanet-where-it-rains-iron-discovered-es',
 'Exoplanet-where-it-rains-iron-discovered-pt',
 'Exoplanet-where-it-rains-iron-discovered-pl',
 'Exoplanet-where-it-rains-iron-discovered-zh-CN',
 'Using-copper-to-prevent-the-spread-of-respiratory-viruses-es',
 'Using-copper-to-prevent-the-spread-of-respiratory-viruses-pt',
 'Using-copper-to-prevent-the-spread-of-respiratory-virus

In [48]:
#refresh already saved articles' html
#for when a design change is implemented.
import time

#for article in articlessofar:
#    tmp = targdic_to_dirfile(articlessofar[article], articlessofar[article]["lang"], elements_dictionary)

In [49]:
import time

# Ensure that the test_enable variable is set to True only for testing
test_enable = False

for article in articles:
    if article not in articleurlssofar or test_enable:
        try:
            dic = sciencedaily_parse_article(article)

            for target_language in languages:
                try:
                    # Processing article in a target language
                    tmp = dic_to_dirfile(dic, target_language, elements_dictionary)
                    articlessofar[tmp[2]] = tmp[1]

                except KeyError:
                    # Handle the case when the required key is not found
                    url_to_dirfile(article, target_language)

                except Exception as e:
                    # Handle other exceptions
                    print(f"Error processing article in {target_language}: {e}")

            # Add the article URL to the list of processed articles
            articleurlssofar.append(article)

        except Exception as e:
            print(f"Error parsing article: {e}")


Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <urlopen error unknown url type: ('https>
Error parsing article: <u

In [50]:
save_obj(headlinessofar,     "headlinessofar")
save_obj(articlessofar,       "articlessofar")
save_obj(articleurlssofar, "articleurlssofar")

In [51]:
def img_thumb_url(key):
    return re.sub("img/", "img/thumbnails/", articlessofar[key]["imgaddr"])

In [61]:
from datetime import date
import os

def form_index(target_language):
    # Read the HTML template
    with open(SUBFOLDER + "templates/index.html", 'r') as template_file:
        html = template_file.read()

    hoy = date.today()
    formatted_date = hoy.strftime("%d/%m/%Y")

    # Dictionary to map placeholders to their replacements
    replacements = {
        "$$date%%": formatted_date,
        "$$home%%": f"{elements_dictionary[f'homepage-{target_language}']}",
        "$$archive%%": f"{elements_dictionary[f'archive-{target_language}']}",
        "$$about%%": f"{elements_dictionary[f'about us-{target_language}']}",
        "$$jp-translation%%": f"{elements_dictionary[f'scientific turk-{target_language}']}",
        "$$latest-articles%%": f"{elements_dictionary[f'latest-articles-{target_language}']}",
        "$$slogan1%%": slogans[0],
        "$$slogan2%%": slogans[1],
        "$$slogan3%%": slogans[2],
        "$$date%%": formatted_date,
        "$$homepage-text%%": f"{elements_dictionary[f'hometext-{target_language}']}",
        "$$see-more%%": f"{elements_dictionary[f'see-more-{target_language}']}",
        "$$target-language%%": target_language
    }

    # Sort articles by date and language
    sorted_articles = sorted(articlessofar.keys(), key=lambda x: articlessofar[x]['date'], reverse=True)
    language_articles = [key for key in sorted_articles if articlessofar[key]["lang"] == target_language]

    # Limit the number of articles to display (22 in this case)
    num_articles_to_display = min(len(language_articles), 22)

    for i, key in enumerate(language_articles[:num_articles_to_display], start=1):
        replacements[f"$$article-link{i}%%"] = articlessofar[key]["pathfromlang"][1:]
        replacements[f"$$headline{i}%%"] = articlessofar[key]["headline"]

        img_url = articlessofar[key]["imgaddr"]
        if img_url[:2] == "..":
            img_url = img_thumb_url(key)[6:]
        replacements[f"$$headline-img{i}%%"] = img_url

    # Replace placeholders in the HTML template
    for placeholder, value in replacements.items():
        html = html.replace(placeholder, value)

    return html

def refresh_indices():
    for language in languages:
        htmlx = form_index(language)
        path = language + "/index.html"

        # Create the directories and write the HTML files
        os.makedirs(os.path.join(SUBFOLDER, language), exist_ok=True)
        with open(os.path.join(SUBFOLDER, path), "w+") as file:
            file.write(htmlx)


In [62]:
import os

def form_archive(target_language):
    # Read the HTML template
    with open(SUBFOLDER + "templates/archive.html", 'r') as template_file:
        html = template_file.read()

    from datetime import date

    hoy = date.today()
    formatted_date = hoy.strftime("%d/%m/%Y")

    # Dictionary to map placeholders to their replacements
    replacements = {
        "$$date%%": formatted_date,
        "$$home%%": elements_dictionary[f"homepage-{target_language}"],
        "$$archive%%": elements_dictionary[f"archive-{target_language}"],
        "$$about%%": elements_dictionary[f"about us-{target_language}"],
        "$$jp-translation%%": elements_dictionary[f"scientific turk-{target_language}"],
        "$$latest-articles%%": elements_dictionary[f"latest-articles-{target_language}"],
        "$$homepage-text%%": elements_dictionary[f"hometext-{target_language}"],
        "$$slogan1%%": slogans[0],
        "$$slogan2%%": slogans[1],
        "$$slogan3%%": slogans[2],
        "$$date%%": formatted_date,
        "$$target-language%%": target_language,
    }

    # Sort articles by date and language
    sorted_articles = sorted(articlessofar.keys(), key=lambda x: articlessofar[x]['date'], reverse=True)

    archive_text = ""

    for i, key in enumerate(sorted_articles):
        if articlessofar[key]["lang"] == target_language:
            link = articlessofar[key]["pathfromlang"][1:]
            title = articlessofar[key]["headline"]
            date = articlessofar[key]["date"]
            meta = articlessofar[key]["meta"]
            
            article_text = f"<p>{date} <a href=\"{link}\" style=\"font-weight:900;color:chocolate\">{title}</a>\n<br>\n"
            article_text += f"<small style=\"color:gray\">{meta}</small></p>\n<br>\n"

            archive_text += article_text

    # Replace placeholders in the HTML template
    replacements["$$archive-text%%"] = archive_text
    for placeholder, value in replacements.items():
        html = html.replace(placeholder, value)

    return html

def refresh_archives():
    for language in languages:
        htmlx = form_archive(language)
        path = f"{language}/archive.html"

        # Create the directories and write the HTML files
        os.makedirs(os.path.join(SUBFOLDER, language), exist_ok=True)
        with open(os.path.join(SUBFOLDER, path), "w+") as file:
            file.write(htmlx)


In [63]:
def form_about(target_language):
    # Read the HTML template
    with open(SUBFOLDER + "templates/about.html", 'r') as template_file:
        html = template_file.read()

    # Dictionary to map placeholders to their replacements
    replacements = {
        "$$home%%": elements_dictionary[f"homepage-{target_language}"],
        "$$archive%%": elements_dictionary[f"archive-{target_language}"],
        "$$about%%": elements_dictionary[f"about us-{target_language}"],
        "$$jp-translation%%": elements_dictionary[f"scientific turk-{target_language}"],
        "$$slogan1%%": slogans[0],
        "$$slogan2%%": slogans[1],
        "$$slogan3%%": slogans[2],
        "$$date%%": d1,
        "$$about-us%%": elements_dictionary[f"about us-{target_language}"],
        "$$about-text%%": elements_dictionary[f"abouttext-{target_language}"],
        "$$homepage-text%%": elements_dictionary[f"hometext-{target_language}"],
        "$$target-language%%": target_language,
    }

    # Replace placeholders in the HTML template
    for placeholder, value in replacements.items():
        html = html.replace(placeholder, value)

    return html

def refresh_abouts():
    for language in languages:
        htmlx = form_about(language)
        path = f"{language}/about.html"

        # Create the directories and write the HTML files
        os.makedirs(os.path.join(SUBFOLDER, language), exist_ok=True)
        with open(os.path.join(SUBFOLDER, path), "w+") as file:
            file.write(htmlx)


In [64]:
refresh_archives()

In [65]:
refresh_abouts()

In [66]:
refresh_indices()